10. Encrypt the plaintext message using RSA algorithm. Then perform the reverse operation to get original plaintext.

In [11]:
def split_into_blocks(message, block_size):
    """Split string into integer blocks"""
    blocks = []
    lengths = []
    for i in range(0, len(message), block_size):
        block = message[i:i+block_size]
        blocks.append(int(block))
        lengths.append(len(block))
    return blocks, lengths


# RSA parameters
p, q = 47, 71
n = p * q
phi = (p - 1) * (q - 1)
e = 79
d = 1019

print("RSA Parameters:")
print(f"p = {p}, q = {q}")
print(f"n = {n}, phi = {phi}")
print(f"Public key (e, n) = ({e}, {n})")
print(f"Private key (d, n) = ({d}, {n})")
print()

# Numeric message given
message = "6882326879666683"

# Determine block size
block_size = len(str(n)) - 1
print(f"Block Size: {block_size} digits")

# Split message into chunks
chunks_message, chunks_length = split_into_blocks(message, block_size)

print("\nPlaintext Blocks:", chunks_message)

# Encryption: ci = mi^e % n
cipher_blocks = []
print("\nCiphertext Blocks:")
for m in chunks_message:
    c = pow(m, e, n)
    cipher_blocks.append(c)
    print(c, end=" ")
print()

# Decryption: mi = ci^d % n
decrypt_result = ""
print("\nDecrypted Blocks:")
for i in range(len(cipher_blocks)):
    m = pow(cipher_blocks[i], d, n)
    decrypted_str = str(m).zfill(chunks_length[i])  # restore original zero padding
    decrypt_result += decrypted_str
    print(decrypted_str, end=" ")
print()

print("\nFinal Reconstructed Message:", decrypt_result)
print("Decryption Successful ✅" if decrypt_result == message else "Error ❌")


RSA Parameters:
p = 47, q = 71
n = 3337, phi = 3220
Public key (e, n) = (79, 3337)
Private key (d, n) = (1019, 3337)

Block Size: 3 digits

Plaintext Blocks: [688, 232, 687, 966, 668, 3]

Ciphertext Blocks:
1570 2756 2091 2276 2423 158 

Decrypted Blocks:
688 232 687 966 668 3 

Final Reconstructed Message: 6882326879666683
Decryption Successful ✅


### If keys need to generated

In [ ]:
import random
import math

def gcd(a, b):
    """Calculate Greatest Common Divisor"""
    while b:
        a, b = b, a % b
    return a

def mod_inverse(e, phi):
    """Calculate modular inverse using Extended Euclidean Algorithm"""
    def extended_gcd(a, b):
        if a == 0:
            return b, 0, 1
        gcd, x1, y1 = extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return gcd, x, y
    
    gcd, x, y = extended_gcd(e, phi)
    return (x % phi + phi) % phi

def is_prime(n):
    """Simple primality test"""
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

def generate_keypair():
    """Generate RSA public and private key pair"""
    # Choose two small prime numbers for demonstration
    primes = [p for p in range(50, 200) if is_prime(p)]
    p = random.choice(primes)
    q = random.choice(primes)
    while p == q:
        q = random.choice(primes)
    
    # Calculate n and phi(n)
    n = p * q
    phi = (p - 1) * (q - 1)
    
    # Choose e (public exponent)
    e = 65537  # Common choice
    while gcd(e, phi) != 1:
        e = random.randint(3, phi - 1)
    
    # Calculate d (private exponent)
    d = mod_inverse(e, phi)
    
    # Return public key (e, n) and private key (d, n)
    return (e, n), (d, n), p, q

def rsa_encrypt(message, public_key):
    """Encrypt message using RSA public key"""
    e, n = public_key
    return pow(message, e, n)

def rsa_decrypt(ciphertext, private_key):
    """Decrypt message using RSA private key"""
    d, n = private_key
    return pow(ciphertext, d, n)

def string_to_ascii(text):
    """Convert string to list of ASCII values"""
    return [ord(char) for char in text]

def ascii_to_string(ascii_list):
    """Convert list of ASCII values back to string"""
    return ''.join(chr(num) for num in ascii_list)

# Generate RSA key pair
public_key, private_key, p, q = generate_keypair()
e, n = public_key
d, n_private = private_key

print("RSA Key Generation:")
print(f"Prime p: {p}")
print(f"Prime q: {q}")
print(f"n = p × q: {n}")
print(f"Public key (e, n): ({e}, {n})")
print(f"Private key (d, n): ({d}, {n})")
print()



# Test 1: Integer Encryption
print("=== INTEGER ENCRYPTION ===")
integer_message = 42
print(f"Original integer: {integer_message}")

encrypted_int = rsa_encrypt(integer_message, public_key)
print(f"Encrypted: {encrypted_int}")

decrypted_int = rsa_decrypt(encrypted_int, private_key)
print(f"Decrypted: {decrypted_int}")
print(f"Success: {integer_message == decrypted_int}")
print()

In [ ]:
# Test 2: String Encryption (character by character)
print("=== STRING ENCRYPTION ===")
string_message = "HELLO"
print(f"Original string: '{string_message}'")

# Convert string to ASCII values
ascii_values = string_to_ascii(string_message)
print(f"ASCII values: {ascii_values}")
# Encrypt each ASCII value
encrypted_chars = []
for ascii_val in ascii_values:
    if ascii_val < n:  # Make sure ASCII value is less than n
        encrypted_char = rsa_encrypt(ascii_val, public_key)
        encrypted_chars.append(encrypted_char)
    else:
        print(f"Warning: ASCII value {ascii_val} >= n ({n})")

print(f"Encrypted ASCII: {encrypted_chars}")

# Decrypt each encrypted value
decrypted_ascii = []
for encrypted_char in encrypted_chars:
    decrypted_char = rsa_decrypt(encrypted_char, private_key)
    decrypted_ascii.append(decrypted_char)

print(f"Decrypted ASCII: {decrypted_ascii}")

# Convert back to string
decrypted_string = ascii_to_string(decrypted_ascii)
print(f"Decrypted string: '{decrypted_string}'")
print(f"Success: {string_message == decrypted_string}")